In [ ]:
import numpy as np
import itertools
import scipy as sc
import matplotlib.pyplot as plt

from sympy.matrices import Matrix


Spinless Hubbard model is:
$$
\mathbf{H}_{Spinless}=\sum_{i}^{L} \epsilon_{i} \hat{a}_{i}^\dagger \hat{a}_{i}  - t \sum_{\langle i,j \rangle}^{L} \hat{a}_{i}^\dagger\hat{a}_j+ U \sum_{i}^{L} \hat{n}_{i}\hat{n}_{i+1}
$$
Where $\hat{n}_i=\hat{a}^\dagger_i\hat{a}_i$, given $L$ sites and $N$ electrons the Hilbert space is of dimension:    
$$n=\binom{L}{N}$$



Normal 1D Hubbard model is:    
$$
\mathbf{H}_{Spin}=\sum_{i,\sigma}^{L} \epsilon_{i} \hat{a}_{i,\sigma}^\dagger \hat{a}_{i,\sigma}  - t \sum_{i,σ}^{L} \hat{a}_{i,\sigma}^\dagger\hat{a}_{i+1,\sigma}+ U \sum_{i}^{L} \hat{n}_{i,\downarrow}\hat{n}_{i,\uparrow}
$$

## Steps to take
Write function that creates the Hamiltonian matrix, given inputs  [Length_of_chain, $N_\uparrow$, $N_\downarrow$, $U$, $t$, external potential $\{ϵ_i\}$]

1.   Generate basis states, taking combination of spin up and spin downs
2.   Use binary representation of basis state, e.g for a state $|0,1,0,1\rangle \longrightarrow 5$, storing states as integers. 
3.   Calculate matrix entries via fast bitwise operations, as searching through Numpy arrays for hopping terms is complicated and expensive. 




### XOR operation for hopping term Example
have basis states $|0,1,1,0\rangle$ and $|1,0,1,0\rangle$, taking XOR and summing gives $$ sum(|1,1,0,0\rangle)=2 $$
When the sum is equal to 2 we have a hopping term 

$$H_T[|0,1,1,0\rangle,|1,0,1,0\rangle]=-t $$
This is for the spinless case, but is identical for the case with spin.

### AND operation for on-site Coulomb repulsion Example
Will store basis states with the first half of bits representing spin-up and the second representing spin-down, e.g 
$$ |0011 ;1001\rangle$$
Then by taking AND of this states spin separated parts, we get the Coulomb repulsion term
$$sum(0011\quad  AND \quad 1001)= 1 $$



### Hamiltonian generation
$L$ is the length of the chain/ring. $\{ϵ\}$ will be random, uniformly generated.


In [ ]:
max_value=0     #NEEDED for B.C: global value that represents the largest possible integer resulting from spin up/down basis states


def Hubbard_Matrix(L,N_up,N_down,U,t,external_potential):
  number_of_sites=2*L                             # potentially occupied orbitals

  def easy_basis_state(N_up):
    easy="1"*N_up +"0"*(L-N_up)
    global maximum
    maximum=float(2**(L))
    return easy
  
  states_up=[]                                 
  states_down=[]

  ordered_first_up=easy_basis_state(N_up)   
  orderesd_first_down= easy_basis_state(N_down)      

  ###########################################################
  # Now generate the  permutations
  ###########################################################
  ups=itertools.permutations(ordered_first_up)
  downs=itertools.permutations(orderesd_first_down)


  # use python set function for creating unique states
  ups_set=set(ups)
  downs_set=set(downs)
  ups_final=list(ups_set)
  downs_final=list(downs_set)
  #for i in ups_final:
    #print(''.join(i))

  for i in range(len(ups_final)):
        BINARY=''.join(ups_final[i])
        p= int(BINARY,2)
        states_up.append(p)

  for i in range(len(downs_final)):
        BINARY=''.join(downs_final[i])
        p= int(BINARY,2)
        states_down.append(p)

  states_down=np.array(states_down)#.reshape((-1,1))
  states_up=np.array(states_up)#.reshape((-1,1))
  #print(states_up)
  #print(states_down)
  basis_states=np.array(list(itertools.product(states_up,states_down)))
  #print(basis_states)
  dim=len(basis_states)

  H=np.zeros((dim,dim),dtype=np.float64)
  #print(H.shape)

  ###################################################################################
  ###                 U term = number of Coulomb repulsions in given state     ######
  ###################################################################################

  for i in range(len(basis_states)):
    upy,downy=basis_states[i][0],basis_states[i][1]
    #print(np.binary_repr(upy,L))
    #print(np.binary_repr(downy,L))
    coulombs= np.binary_repr(upy & downy,L).count('1')
    #print(coulombs)
    if coulombs>0:
      H[i,i]+= U * coulombs 
  


  ###################################################################################
  ###                 T term = hopping kinetic energy terms                    ######
  ###################################################################################
  for i in range(len(basis_states)):
    for j in range(len(basis_states)):

      if i>j:                                                             ## take XOR's
        hops_up=basis_states[i][0]^basis_states[j][0]
        hops_down=basis_states[i][1]^basis_states[j][1]

        result_up=np.binary_repr(hops_up,L).count('1')                        # count number of hops, we want it to be 2
        result_down=np.binary_repr(hops_down,L).count('1')
        
        if (result_up==0 and result_down==2) or (result_up==2 and result_down==0):
                                                                              # check if hopping is between neighbours #
          check_up=hops_up&(hops_up<<1)                  
          check_down=hops_down&(hops_down<<1)

          neighbour_up=np.binary_repr(check_up,L).count('1')
          neighbour_down=np.binary_repr(check_down,L).count('1')

          if (neighbour_up==1 and neighbour_down==0) or (neighbour_up==0 and neighbour_down==1):
            H[i,j]=H[j,i]= -t

  ########################### NEED TO DO Boundary conditions hopping ##############################

  #################################################################################################
  #              External potential terms 
  #################################################################################################
  for i in range(len(basis_states)):
    UP=basis_states[i][0]
    DOWN=basis_states[i][1]
    UP_bin=[int(x) for x in np.binary_repr(UP,L)]
    DOWN_bin=[int(x) for x in np.binary_repr(DOWN,L)]
    product= external_potential*(np.array(UP_bin) +np.array(DOWN_bin))
    #print(DOWN_bin)
    #print(UP_bin)
    #print(product)
    H[i,i]+= sum(product)



  print(H)  

    
  



  

L=4
external_pot=[0.2,-0.2,0.3,0.1]
Hubbard_Matrix(L,1,1,1,8,external_pot)#np.random.uniform(-0.1,0.1,L))





[[ 1.2  0.  -8.   0.   0.   0.   0.   0.  -8.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.3  0.  -8.   0.   0.   0.   0.   0.  -8.   0.   0.   0.   0.
   0.   0. ]
 [-8.   0.   0.4 -8.   0.   0.   0.   0.   0.   0.  -8.   0.   0.   0.
   0.   0. ]
 [ 0.  -8.  -8.  -0.1  0.   0.   0.   0.   0.   0.   0.  -8.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.3  0.  -8.   0.   0.   0.   0.   0.  -8.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   1.4  0.  -8.   0.   0.   0.   0.   0.  -8.
   0.   0. ]
 [ 0.   0.   0.   0.  -8.   0.   0.5 -8.   0.   0.   0.   0.   0.   0.
  -8.   0. ]
 [ 0.   0.   0.   0.   0.  -8.  -8.   0.   0.   0.   0.   0.   0.   0.
   0.  -8. ]
 [-8.   0.   0.   0.   0.   0.   0.   0.   0.4  0.  -8.   0.  -8.   0.
   0.   0. ]
 [ 0.  -8.   0.   0.   0.   0.   0.   0.   0.   0.5  0.  -8.   0.  -8.
   0.   0. ]
 [ 0.   0.  -8.   0.   0.   0.   0.   0.  -8.   0.   1.6 -8.   0.   0.
  -8.   0. ]
 [ 0.   0.   0.  -8.   0.   0.   0.   0.   0.  -8.  -8.   0.1  0.   0.
   0.

In [ ]:
print(7&(7<<1))
k=7<<1
print(k)
print(7&k)

6
14
6


In [ ]:
print(bin(7))
np.array([int(x) for x in np.binary_repr(8,4)])*np.full((1,4),2)

0b111


array([[2, 0, 0, 0]])